In [ ]:
import sys

assert sys.version_info >= (3, 7)
from packaging import version
import tensorflow as tf
assert version.parse(tf.__version__) >= version.parse("2.8.0")
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
tf.keras.utils.get_file(
    "ridership.tgz",
    "https://github.com/ageron/data/raw/main/ridership.tgz",
    cache_dir=".",
    extract=True
)
path = Path("datasets/ridership/CTA_-_Ridership_-_Daily_Boarding_Totals.csv")
df = pd.read_csv(path, parse_dates=["service_date"])
df.columns = ["date", "day_type", "bus", "rail", "total"]  # shorter names
df = df.sort_values("date").set_index("date")
df = df.drop("total", axis=1)  # no need for total, it's just bus + rail
df = df.drop_duplicates()  # remove duplicated months (2011-10 and 2014-07)

108512/108512 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
def to_windows(dataset, length):
    dataset = dataset.window(length, shift=1, drop_remainder=True)
    return dataset.flat_map(lambda window_ds: window_ds.batch(length))

In [ ]:
df_mulvar = df[["bus", "rail"]] / 1e6  # use both bus & rail series as input
df_mulvar["next_day_type"] = df["day_type"].shift(-1)  # we know tomorrow's type
df_mulvar = pd.get_dummies(df_mulvar)  # one-hot encode the day type
mulvar_train = df_mulvar["2016-01":"2018-12"]
mulvar_valid = df_mulvar["2019-01":"2019-05"]
mulvar_test = df_mulvar["2019-06":]

tf.random.set_seed(42)  # extra code – ensures reproducibility
seq_length = 56
# Convert all columns to numeric type before creating the dataset
# Convert all columns to numeric type, ensuring they are standard float types (float32 or float64)


In [ ]:
my_series = tf.data.Dataset.range(7)
dataset = to_windows(to_windows(my_series, 3), 4)
list(dataset)

[<tf.Tensor: shape=(4, 3), dtype=int64, numpy=
 array([[0, 1, 2],
        [1, 2, 3],
        [2, 3, 4],
        [3, 4, 5]])>,
 <tf.Tensor: shape=(4, 3), dtype=int64, numpy=
 array([[1, 2, 3],
        [2, 3, 4],
        [3, 4, 5],
        [4, 5, 6]])>]

In [ ]:
dataset = dataset.map(lambda S: (S[:, 0], S[:, 1:]))
list(dataset)

[(<tf.Tensor: shape=(4,), dtype=int64, numpy=array([0, 1, 2, 3])>,
  <tf.Tensor: shape=(4, 2), dtype=int64, numpy=
  array([[1, 2],
         [2, 3],
         [3, 4],
         [4, 5]])>),
 (<tf.Tensor: shape=(4,), dtype=int64, numpy=array([1, 2, 3, 4])>,
  <tf.Tensor: shape=(4, 2), dtype=int64, numpy=
  array([[2, 3],
         [3, 4],
         [4, 5],
         [5, 6]])>)]

In [ ]:
def to_seq2seq_dataset(series, seq_length=56, ahead=14, target_col=1,
                       batch_size=32, shuffle=False, seed=None):
    # Convert all columns in the series to numeric type before creating the dataset
    # Explicitly convert to float32 to ensure compatibility with TensorFlow
    series = series.astype(np.float32)  # Convert all columns to float32

    ds = to_windows(tf.data.Dataset.from_tensor_slices(series.values), ahead + 1) # Pass values to avoid index issues
    ds = to_windows(ds, seq_length).map(lambda S: (S[:, 0], S[:, 1:, 1]))
    if shuffle:
        ds = ds.shuffle(8 * batch_size, seed=seed)
    return ds.batch(batch_size)

In [ ]:
seq2seq_train = to_seq2seq_dataset(mulvar_train, shuffle=True, seed=42)
seq2seq_valid = to_seq2seq_dataset(mulvar_valid)

NameError: name 'to_seq2seq_dataset' is not defined

In [ ]:
def fit_and_evaluate(model, train_set, valid_set, learning_rate, epochs=500):
    early_stopping_cb = tf.keras.callbacks.EarlyStopping(
        monitor="val_mae", patience=50, restore_best_weights=True)
    opt = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
    model.compile(loss=tf.keras.losses.Huber(), optimizer=opt, metrics=["mae"])
    history = model.fit(train_set, validation_data=valid_set, epochs=epochs,
                        callbacks=[early_stopping_cb])
    valid_loss, valid_mae = model.evaluate(valid_set)
    return valid_mae * 1e6

# LSTM

In [ ]:
tf.random.set_seed(42)  # extra code – ensures reproducibility
lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(32, return_sequences=True, input_shape=[None, 5]),
    tf.keras.layers.Dense(14)
])
fit_and_evaluate(lstm_model, seq2seq_train, seq2seq_valid,
                 learning_rate=0.1, epochs=30)

Epoch 1/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 0.0862 - mae: 0.3397 - val_loss: 0.0190 - val_mae: 0.1591
Epoch 2/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.0178 - mae: 0.1646 - val_loss: 0.0176 - val_mae: 0.1433
Epoch 3/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0157 - mae: 0.1518 - val_loss: 0.0165 - val_mae: 0.1403
Epoch 4/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0149 - mae: 0.1482 - val_loss: 0.0158 - val_mae: 0.1355
Epoch 5/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.0141 - mae: 0.1430 - val_loss: 0.0151 - val_mae: 0.1321
Epoch 6/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - loss: 0.0135 - mae: 0.1383 - val_loss: 0.0144 - val_mae: 0.1289
Epoch 7/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - loss: 0.0129 - mae: 0.1357 - val_loss: 0.0138 - val_mae: 0.1254
Epoch 8/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0124 - mae: 0.1316 - val_loss: 0.0132 - val_mae: 0.1225
Epoch 9/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.011

99118.04646253586

# GRU

In [ ]:
tf.random.set_seed(42)  # extra code – ensures reproducibility
gru_model = tf.keras.Sequential([
    tf.keras.layers.GRU(32, return_sequences=True, input_shape=[None, 5]),
    tf.keras.layers.Dense(14)
])
fit_and_evaluate(gru_model, seq2seq_train, seq2seq_valid,
                 learning_rate=0.1, epochs=30)

Epoch 1/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 5s 68ms/step - loss: 0.1367 - mae: 0.4191 - val_loss: 0.0184 - val_mae: 0.1648
Epoch 2/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - loss: 0.0172 - mae: 0.1515 - val_loss: 0.0161 - val_mae: 0.1358
Epoch 3/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - loss: 0.0137 - mae: 0.1368 - val_loss: 0.0145 - val_mae: 0.1278
Epoch 4/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - loss: 0.0123 - mae: 0.1292 - val_loss: 0.0133 - val_mae: 0.1214
Epoch 5/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.0113 - mae: 0.1215 - val_loss: 0.0124 - val_mae: 0.1168
Epoch 6/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - loss: 0.0106 - mae: 0.1158 - val_loss: 0.0116 - val_mae: 0.1137
Epoch 7/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - loss: 0.0102 - mae: 0.1127 - val_loss: 0.0112 - val_mae: 0.1116
Epoch 8/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - loss: 0.0097 - mae: 0.1097 - val_loss: 0.0107 - val_mae: 0.1099
Epoch 9/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 0.009

90826.6231417656